# SOP生成作业
_2020201623_
_2023/3/19_

## 注意事项
1. 请确保对应写入docx和pdf的文件夹为空，以防止可能的读写问题
2. pdf的生成速度相较于docx更慢，没必要等待pdf全部生成
3. 样例已存储于pos文件夹下pdf_Ex和word_Ex两文件夹中

## 准备工作

In [1]:
#下述安装包对应爬虫功能
#!pip install requests
#!pip install beautifulsoup4

#下述安装包对应文件输出功能
#!pip install docxtpl
#!pip install docx2pdf

#为了在jupyter notebook正常使用docx2pdf
#!pip install pywin32

In [15]:
#下述import对应爬虫功能
import requests as rq
import bs4
import csv
from bs4 import BeautifulSoup

#下述import对应文件字符替换及输出功能
from docxtpl import DocxTemplate
from docx2pdf import convert
import pandas as pd
import os
import win32com.client
import time

## 学校爬取，简历填充及输出

In [3]:
#爬虫
def Sprider(url):

    # 发送请求，r为页面响应
    r = rq.get(url)

    # r.text获取页面代码
    # 使用lxml解析页面代码
    #Ctrl+Shift+I查看页面代码,本例所需要的是table一栏下class=shorttop的<a>数据
    soup = BeautifulSoup(r.text, 'lxml')
    s = soup.find('table',class_ = 'shorttop')
    content = s.find_all('a')
    
    #生成排名列表
    rank_list = []

    #抓取排名前30的学校
    count = 0
    for dep in content:
        d = {}
        temp_text = dep.text.split(',') #数据以逗号分隔，形式Department of economics，Havard University。故分割后仅保留最后一位，以保留学校
        d = temp_text[-1]
        if dep.text != '':
            rank_list.append(d)
            count += 1
            if count >= 30:
                break

    return rank_list

In [16]:
# sop--statement of purpose
#简历文字替换及填充
class sop:
    def __init__(self):
        self.url = 'https://ideas.repec.org/top/top.econdept.html'
        self.school_list = Sprider(self.url)
        self.department_list =  ["economics department", "business school", "statistics department", "mathematics deparment","school of public policy"]
        self.z_path = 'D:\Library\spring_backup\AI\HW' #主文件夹路径
        self.s_path = 'D:\Library\spring_backup\AI\HW\sop\word' #输出文件路径
        self.pdf_path = 'D:\Library\spring_backup\AI\HW\sop\pdf'
    #替换及输出docx文档   
    def output_docx(self):

        tpl = DocxTemplate(self.z_path+r"\\template\\template2.docx") #给定模板

        for depart in self.department_list:
            for school in self.school_list:
                context = {
                    "institution": school,
                    "department":  depart
                }
                tpl = DocxTemplate(self.z_path+r"\\template\\template2.docx")
                tpl.render(context)
                tpl.save(self.s_path+r"\\"+depart+","+school+".docx")   
    
    #转换文件至pdf
    def convert2pdf(self):
        word_path =self.s_path
        word_to_pdf = self.pdf_path
        for root, dirs, files in os.walk(word_path):
            for name in files:
                convert(word_path+"/"+ name, word_to_pdf+"/"+name.replace("docx","pdf"))
                time.sleep(2) #让程序有时间把文件关掉


In [17]:
Sop = sop()
Sop.output_docx()
Sop.convert2pdf()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 参考


-[1]: https://zhuanlan.zhihu.com/p/320314207 (python办公自动化（一）DocxTemplate批量生成word)
-[2]:https://www.cnblogs.com/awesometang/p/11991755.html (【爬虫教程】吐血整理，最详细的爬虫入门教程~)
-[3]:https://www.geeksforgeeks.org/python-web-scraping-tutorial/ (Python Web Scraping Tutorial)
-[4]:https://blog.csdn.net/Xusansna/article/details/124061023 (docx2pdf转换问题)
-[5]:https://blog.csdn.net/qq_44081582/article/details/117962462 (jupyter notebook快捷键及标记的使用)
